In [1]:
(ql:quickload "kenzo")
(in-package :cat)

To load "kenzo":
  Load 1 ASDF system:
    kenzo
; Loading "kenzo"



#<PACKAGE "CAT">

# Objects with effective homology

## Introduction

The present chapter describes the set of programming tools for handling *objects with effective homology*. The theoretical material may be found in the paper *Constructive Algebraic Topology* by Julio Rubio Garcia and Francis Sergeraert (Available at the web site
[**https://www-fourier.ujf-grenoble.fr/~sergerar/**](https://www-fourier.ujf-grenoble.fr/~sergerar/)). The terminology used in this chapter is compatible with this reference. The terminology used in this chapter is compatible with this reference.

## Reduction

A *reduction* in `Kenzo` is a 5-tuple $(\hat C, C, f, g, h)$:
$$
\begin{aligned}
\hat{C}\quad & \qquad\stackrel{h}{\longrightarrow} & \mbox{$^s\hat{C}$} \\
{\scriptstyle f} \downarrow \uparrow {\scriptstyle g} & & \\
C\quad & &
\end{aligned}
$$
where $\hat C$ and $C$ are chain complexes, $f$ and $g$  chain complex morphisms and
$h$ a homotopy operator. Hereafter, $\hat C$ is called the *top chain complex* and $C$ the *bottom chain complex*. ${}^s\hat C$ is $\hat C$ shifted, i.e. $h$ has degree $1$. The mappings $f$, $g$, $h$, together with the differential operator $d$ on $\hat C$, must verify the following relations:
$$
\begin{aligned}
f \circ g & = & 1_C \\
h \circ d + d \circ h & = & 1_{\hat C} - g \circ f \\
f \circ h & = & 0   \\
h \circ g & = & 0   \\
h \circ h & = & 0 
\end{aligned}
$$
The morphisms $f$ and $g$ and the homotopy operator $h$ describe the (big) chain complex $\hat C$ as the direct sum
$$ \hat C= \hat C_1 \oplus \hat C_2$$
where $\hat C_1 = Im(g) \simeq C$ and  $\hat C_2= Ker(f)$. The summands $\hat{C}_1$ and $\hat{C}_2$ are sub-chain complexes of $\hat{C}$ and the second one is acyclic, provided with an explicit contraction, namely the restriction of $h$ to $\hat{C}_2$.


### Representation of a reduction

A reduction is implemented as an instance of the `CLOS` class `REDUCTION`, whose definition is:

```commonlisp
(DEFCLASS REDUCTION ()
     ;; Top Chain Complex
    ((tcc :type chain-complex :initarg :tcc :reader tcc1)
     ;; Bottom Chain Complex
     (bcc :type chain-complex :initarg :bcc :reader bcc1)
     (f :type morphism :initarg :f :reader f1)
     (g :type morphism :initarg :g :reader g1)
     (h :type morphism :initarg :h :reader h1)
     ;; IDentification NuMber
     (idnm :type fixnum :initform (incf *idnm-counter*) :reader idnm)
     ;; ORiGiN
     (orgn :type list :initarg :orgn :reader orgn)))
```

This class has 7 slots:

1. `tcc`, the object of type `chain-complex` representing the chain complex $\hat C$ (top chain complex).
2. `bcc`,  the object of type `chain-complex` representing the chain complex $C$ (bottom chain complex).
3. `f`, the object of type `morphism` representing the morphism $f$.
4. `g`, the object of type `morphism` representing the morphism $g$.
5. `h`, the object of type `morphism`  representing the morphism $h$.
6. `idmn`, an integer, number plate for the object.
7. `orgn`, a comment list carefully chosen.

The accessors of the slots are the functions whose name appears after the specifier `:reader` in the class definition. A printing method has been associated to the class `REDUCTION` and the external representation of an instance is a string like `[Kn Reduction K n' => Kn'']`, where `n` is the identification number of the reduction, `n'` (resp. `n''`) is the identification number of the top (resp. bottom) chain complex.


### The function `build-rdct`

To facilitate the construction of instances of the `REDUCTION` class, the
software provides the function `build-rdct`.

```commonlisp
build-rdct :f f :g g :h h :orgn orgn
```

defined with keyword parameters. The returned value is an instance of the class `REDUCTION`. 
The keyword arguments are:

- *f*, the object of type `morphism` representing the morphism $f$.
- *g*, the object of type `morphism` representing the morphism $g$.
- *h*, the object of type `morphism` representing the morphism $h$.
- *orgn*, the comment list carefully chosen since the system does not build a new instance of the class if it finds in the list of already built reductions, `*rdct-list*`, a reduction  with the same comment list.

The `tcc` slot and the `bcc` slot of the instance are taken respectively from the `sorc` slot (source slot) and the `trgt` slot (target slot) of the morphism $f$. The function `build-rdct` controls the validity of the degrees of the morphisms and pushes the new created instance on the list `*rdct-list*`.

### Useful macros and functions

`(cat-init)` *[Function]*
> Clear among others, the list `*rdct-list*`, list of user created reductions  and reset the global counter to 1. See also the description of this function in chapter 1.

`(rdct` *n*`)` *[Function]*
> Retrieve in the list `*rdct-list*` the reduction instance whose identification (as `Kenzo` object) is *n*. If it does not exist, return `NIL`.

`(bcc` *rdct* `&rest` *args*`)` *[Macro]*
> With only one argument (a reduction *rdct*) this macro selects the bottom chain complex of the reduction. Otherwise, it applies  the differential of the bottom chain complex of the reduction *rdct* on the arguments *args*, (either *degree generator* or *cmb*).

`(tcc` *rdct* `&rest` *args*`)` *[Macro]*
> With only one argument (a reduction *rdct*) this macro selects the top chain complex of the reduction. Otherwise, it applies  the differential of the top chain complex of the reduction *rdct* on the arguments *args*, (either *degree generator* or *cmb*).

`(f` *rdct* `&rest` *args*`)` *[Macro]*
> With only one argument (a reduction *rdct*) this macro selects the morphism $f$ of the reduction. Otherwise, it applies the morphism $f$ of the reduction *rdct* on the arguments *args*, (either *degree generator* or *cmb*).

`(g` *rdct* `&rest` *args*`)` *[Macro]*
> With only one argument (a reduction *rdct* this macro selects the morphism $g$ of the reduction. Otherwise, it applies  the morphism $g$ of the reduction *rdc* on the arguments *args*, (either *degree generator* or *cmb*).

`(h` *rdct* `&rest` *args*`)` *[Macro]*
> With only one argument (a reduction *rdct*) this macro selects the morphism $h$ of the reduction. Otherwise, it applies the morphism $h$ of the reduction *rdc* on the arguments *args*, (either *degree generator* or *cmb*).

`(trivial-rdct` *chcm*`)` *[Function]*
> Build the trivial reduction involving only the chain complex chcm, as shown in the following diagram,
$$
\begin{aligned}
 C \quad & \quad\stackrel{Zero} {\longrightarrow} & {}^s C \\
 {\scriptstyle {Id}} \downarrow \uparrow {\scriptstyle {Id}} & &  \\
  C \quad & &
\end{aligned}
$$
where the morphism *Zero* is the zero morphism of degree 1 in the chain complex $C$ and *Id* is the identity morphism in that chain complex (see the functions `zero-mrph` and `idnt-mrph` in the chain complex chapter).

`(cmps` *brdct trdct*`)` *[Method]*
> Build a new reduction from the two reductions $b$ and $t$ (here, respectively *brdct* and *trdct*). This is done by a call to `build-rdct` with the following parameters:
$$
\begin{aligned}
f & = & f_{b}\circ f_{t}, \\
g & = & g_{t}\circ g_{b}, \\
h & = &  h_{t}  + g_{t} \circ h_{b} \circ f_{t}.
\end{aligned}
$$
The compositions and additions of morphisms are realized respectively by the methods `cmps`, `i-cmps` and `add` (see chapter 1). We recall that the `tcc` and the `bcc` slots of this new created reduction are respectively the source and target slots of the new morphim $f$.

### Verification functions

The two following functions are very helpful to verify the coherence of the the various mappings involved in a reduction. Let us recall the diagram of a reduction:
$$
\begin{aligned}
\hat{C} \quad & \quad\stackrel{h}{\longrightarrow} & \mbox{$^s\hat{C}$} \\
{\scriptstyle f} \downarrow \uparrow {\scriptstyle g} & & \\
C \quad & &
\end{aligned}
$$

`(pre-check-rdct` *rdct*`)` *[Function]*
> Assign to the following lisp global variables, the morphism instances computed from the morphisms of the reduction *rdct*, according to the  formulas:
$$
\begin{aligned}
  \mathtt{*tdd*} & = & d_{\hat C} \circ d_{\hat C},\\
  \mathtt{*bdd*} & = & d_C \circ d_C , \\
  \mathtt{*id-fg*} & = &  f \circ g - Id_C, \\
  \mathtt{*id-gf-dh-hd*} & = & Id_{\hat C} - g \circ f - (d_{\hat C} \circ h + h \circ d_{\hat C}),\\
  \mathtt{*hh*} & = & h \circ h, \\
  \mathtt{*fh*} & = & f \circ h, \\
  \mathtt{*hg*} & = & h \circ g, \\
  \mathtt{*df-fd*} & = & d_C \circ f - f \circ d_{\hat C}, \\
  \mathtt{*dg-gd*} & = & d_{\hat C} \circ g - g \circ d_C.
\end{aligned}
$$
In these formulas  $d_C$ and $d_{\hat C}$ are the respective boundary operators of the
chain complexes $C$ and $\hat C$. $Id_C$ and $Id_{\hat C}$ are the identity morphisms
on the chain complexes $C$ and $\hat C$  built by the function `idnt-mrph`

`(check-rdct)` *[Function]*
> Map  all the morphisms prepared by the function *pre-check-rdct* on chosen combinations.
This function having no parameters, the user must  assign to  the two lisp global variables:
`*tc*` and `*bc*`,  valid combinations belonging respectively to $\hat C$ and $C$. The call to this function is simply `(check-rdct)`. This function knows what morphism to apply
to the combinations and pauses after each evaluation to allow the user to inspect each result.
To resume the execution, the user must enter a blank character. If the morphisms are coherent, the result of each mapping is a null combination.

#### Example

To show an example about the verification functions, we first define a locally effective version
of the standard simplex $\Delta^n$. The following function `cdelta` builds the standard simplex
in dimension *dmns*. In using the created chain complex, the user must bear in mind that
the only valid vertices are the vertices numbered from $(0)$ to $(dmns)$.

In [2]:
(defun cdelta (dmns)
    (build-chcm
     :cmpr #'l-cmpr
     :basis :locally-effective
     :bsgn '(0)
     :intr-dffr
         #'(lambda (degr gmsm)
            (make-cmbn
             :degr (1- degr)
             :list (do ((rslt +empty-list+
                              (cons (cons sign
                                          (append
                                           (subseq gmsm 0 nark)
                                           (subseq gmsm (1+ nark))))
                                    rslt))
                        (sign 1 (- sign))
                        (nark 0 (1+ nark)))
                       ((> nark degr) rslt))))
     :strt :gnrt
     :orgn `(locally effective version of C_* delta ,dmns)))

CDELTA

Now, let us define 3 functions `make-f`, `make-g` and `make-h` which build the following respective morphisms between $\Delta^m$ and $\Delta^n$:

- `(make-f tdms bdms)` builds a projection morphism $f$ from $\Delta^{tdms}$ to $\Delta^{bdms}$, $tdms>=bdms$, where the vertices $(0)$ to $(dms)$ of $\Delta^{tdms}$ are applied on the vertices of the same number in $\Delta^{bdms}$ and the vertices $(bdms+1)$ to $(tdms)$ are applied on the vertex $(bdms)$ of $\Delta^{bdms}$.
- `(make-g tdms bdms)` builds the injection morphism $g$ from $\Delta^{bdms}$ to $\Delta^{tdms}$ (the slot `:intr` is the identity function).
- `(make-h tdms bdms)` builds a homotopy morphism of degree 1, $h$, from $\Delta^{tdms}$ to itself, connecting w.r.t. the homotopy relation, the chain map $g \circ f$ with the identity morphism $Id$.

In [3]:
(defun make-f (tdmns bdmns)
    (build-mrph
     :sorc (cdelta tdmns) :trgt (cdelta bdmns) :degr 0
     :intr #'(lambda (degr gmsm)
                (let ((pos (position-if #'(lambda (vertex) (>= vertex bdmns))
                                        gmsm)))
                     (if pos
                         (if (< pos degr)
                             (zero-cmbn degr)
                             (cmbn degr 1 (nconc (butlast gmsm) (list bdmns))))
                         (cmbn degr 1 gmsm))))
     :strt :gnrt
     :orgn `(projection delta ,tdmns => delta ,bdmns)))

MAKE-F

In [4]:
(defun make-g (tdmns bdmns)
    (build-mrph
     :sorc (cdelta bdmns) :trgt (cdelta tdmns) :degr 0
     :intr #'identity
     :strt :cmbn
     :orgn `(injection delta ,bdmns => delta ,tdmns)))

MAKE-G

In [5]:
(defun make-h (tdmns bdmns)
    (build-mrph
     :sorc (cdelta tdmns) :trgt (cdelta tdmns) :degr +1
     :intr #'(lambda (degr gmsm)
                (let ((pos (position-if #'(lambda (vertex) (>= vertex bdmns))
                                        gmsm)))
                     (if pos
                         (if (member bdmns gmsm)
                             (zero-cmbn (1+ degr))
                             (cmbn (1+ degr) (-1-expt-n pos)
                                       (append (subseq gmsm 0 pos) (list bdmns)
                                               (subseq gmsm pos))))
                         (zero-cmbn (1+ degr)))))
     :strt :gnrt
     :orgn `(homotopy for delta ,tdmns => ,bdmns)))

MAKE-H

We may now define a function to build a reduction. One has not to define a priori the standard
simplices: this is done  in  all the functions and we know that there is no duplication, because
before creation, the system checks, owing to the comment list, if the instance of a class (here a chain complex) already exists.

In [6]:
(defun make-rdct (tdmns bdmns)
    (let ((rdct (build-rdct
                 :f (make-f tdmns bdmns)
                 :g (make-g tdmns bdmns)
                 :h (make-h tdmns bdmns)
                 :orgn `(reduction delta ,tdmns ,bdmns))))
         rdct))

MAKE-RDCT

We now build with a simple call to the function `make-rdct` the chain complexes corresponding 
to $\Delta^6$ and $\Delta^3$, the 3 morphisms $f$, $g$, $h$ and the reduction. The lisp utility function `inspect` gives an idea of the organisation of the class instance. Then we verify the coherency of the reduction with the functions `pre-check-rdct` and `check-rdct`.

In [7]:
(def rdct (make-rdct 6 3))

[K8 Reduction K1 => K3]

In [8]:
(inspect rdct)


The object is a STANDARD-OBJECT of type REDUCTION.
0. TCC: [K1 Chain-Complex]
1. BCC: [K3 Chain-Complex]
2. F: [K5 Morphism (degree 0): K1 -> K3]
3. G: [K6 Morphism (degree 0): K3 -> K1]
4. H: [K7 Morphism (degree 1): K1 -> K1]
5. IDNM: 8
6. ORGN: (REDUCTION DELTA 6 3)
> 

NIL

In [9]:
(orgn rdct)

(REDUCTION DELTA 6 3)

In [10]:
(pre-check-rdct rdct)


---done---

NIL

In [11]:
(setf *tc* (cmbn 2 1 '(0 1 2) 10 '(1 2 3) 100 '(1 2 4) 1000 '(2 3 4)))


----------------------------------------------------------------------{CMBN 2}
<1 * (0 1 2)>
<10 * (1 2 3)>
<100 * (1 2 4)>
<1000 * (2 3 4)>
------------------------------------------------------------------------------


In [12]:
(setf *bc* (cmbn 3 4 '(0 1 2 3)))


----------------------------------------------------------------------{CMBN 3}
<4 * (0 1 2 3)>
------------------------------------------------------------------------------


In this documentation, we use the non-interactive function `check-rdct-no-wait` which does the whole check at once without waiting the newline input between two elementary checks.

In [13]:
(check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 2}
<1 * (0 1 2)>
<10 * (1 2 3)>
<100 * (1 2 4)>
<1000 * (2 3 4)>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<4 * (0 1 2 3)>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 0}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DG-GD* = 0
R

NIL

We do now the same thing with a more complicated reduction which is the
composition of the two following reductions:

In [14]:
(def trdct (make-rdct 6 4))

[K35 Reduction K1 => K30]

In [15]:
(def brdct (make-rdct 4 3))

[K39 Reduction K30 => K3]

In [16]:
(setf rdct (cmps brdct trdct))

[K45 Reduction K1 => K3]

In [17]:
(pre-check-rdct rdct)


---done---

NIL

In [18]:
(check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 2}
<1 * (0 1 2)>
<10 * (1 2 3)>
<100 * (1 2 4)>
<1000 * (2 3 4)>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<4 * (0 1 2 3)>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 0}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DG-GD* = 0
R

NIL

## Homotopy equivalence

A *homotopy equivalence* between two chain complexes $C$ and $EC$ is a pair of reductions:

$$
\begin{aligned}
  & \qquad\hat{C} & \\
 \rho_1\swarrow\nearrow   & & \nwarrow\searrow \rho_2 \\
C \quad & & EC\quad
\end{aligned}
$$

If $C$ and $EC$ are *free* $\mathbb{Z}$-chain complexes, a usual chain equivalence between them can be
organized in this way. Frequently the chain complexes $C$ and $\hat C$ are *locally effective*
and on the contrary, the chain complex $EC$ is *effective*, so that $EC$ can be understood
as a description of the homology of $C$. More precisely, $EC$ is a tool allowing one to compute the
homology of $C$. The chain complexe $\hat C$ is only an intermediate object.

### Representation of a homotopy equivalence

A homotopy equivalence is implemented as an instance of the `CLOS` class `HOMOTOPY-EQUIVALENCE`,
whose definition is

```commonlisp
(DEFCLASS HOMOTOPY-EQUIVALENCE ()
     ;; Left Bottom Chain Complex
    ((lbcc :type chain-complex :initarg :lbcc :reader lbcc1)
     ;; Top Chain Complex
     (tcc  :type chain-complex :initarg :tcc  :reader tcc1)
     ;; Bottom Right Chain Complex
     (rbcc :type chain-complex :initarg :rbcc :reader rbcc1)
     ;; Left f
     (lf   :type morphism      :initarg :lf   :reader lf1)
     ;; Left g
     (lg   :type morphism      :initarg :lg   :reader lg1)
     ;; Left h
     (lh   :type morphism      :initarg :lh   :reader lh1)
     ;; Right f
     (rf   :type morphism      :initarg :rf   :reader rf1)
     ;; Right g
     (rg   :type morphism      :initarg :rg   :reader rg1)
     ;; Right h
     (rh   :type morphism      :initarg :rh   :reader rh1)
     ;; Left ReDuCTion
     (lrdct :type reduction    :initarg :lrdct :reader lrdct)
     ;; Right ReDuCTion
     (rrdct :type reduction    :initarg :rrdct :reader rrdct)
     ;; IDentification NuMber
     (idnm :type fixnum :initform (incf *idnm-counter*) :reader idnm)
     (orgn :type list          :initarg :orgn :reader orgn)))
```

This class has 13 slots:

- `lbcc`, the object of type `chain complex` representing the chain complex $C$  (**l**eft **b**ottom **c**hain **c**omplex on the diagram).
- `tcc`,  the object of type `chain complex`  representing the chain complex $\hat C$ (**t**op **c**hain **c**omplex).
- `rbcc`, the object of type `chain complex`  representing the chain complex $EC$ (**r**ight **b**ottom **c**hain **c**omplex on the diagram).
- `lf`, the object of type `morphism` representing the morphism $f$ of the (**l**eft) reduction $\rho_1$.
- `lg`, the object of type `morphism` representing the morphism $g$ of the (**l**eft) reduction $\rho_1$.
- `lh`, the object of type `morphism`  representing the morphism $h$ of the (**l**eft) reduction $\rho_1$.
- `rf`, the object of type `morphism`  representing the morphism $f$ of the (**r**ight) reduction $\rho_2$.
- `rg`, the object of type `morphism` representing the morphism $g$ of the (**r**ight) reduction $\rho_2$.
- `rh`, the object of type `morphism` representing the morphism $h$ of the (**r**ight) reduction $\rho_2$.
- `lrdc`, the object of type `reduction` representing the (**l**eft) reduction $\rho_1$.
- `rrdc`, the object of type `reduction` representing the (**r**ight) reduction $\rho_2$.
- `idn`, an integer, number plate for the object, set by the system.
- `orgn`, an adequate comment list.

When an instance is created, the  printing method associated to the class `HOMOTOPY EQUIVALENCE` prints a string like `[Kn Homotopy-Equivalence Ka <= Kb => Kc]`, where `n` is the number plate of the equivalence, `a` is the id number of $C$, `b` the id number of $\hat C$, `c` the id number of $EC$.


### The function `build-hmeq`

To facilitate the construction of instances of the `HOMOTOPY-EQUIVALENCE` class, the program provides the function, (in fact a method), `build-hmeq` which may be used in the following ways:
either with the two reductions $\rho_1$ and $\rho_2$ or explicitly with the morphisms.
The selection of the adequate method is done by inspecting the first keyword, which is 
`:lrdct` in the first case and `:lf` in the second one. In both cases the returned value is an instance of the class `HOMOTOPY-EQUIVALENCE`.

1) `(build-hmeq :lrdct` *lrdct* `:rrdct` *rrdc* `:orgn` *orgn*`)` *[Method]*
> The keyword arguments are:
- *lrdct*, the object of type `reduction` representing the reduction $\rho_1$.
- *rrdct*, the object of type `reduction` representing the reduction $\rho_2$.
- *org*, the comment list.
The function `build-heq-from-rdc` calls internally the standard constructor `make-instance`. All the needed information is contained in the two reductions.

2) `(build-hmeq :lf` *lf* `:lg` *lg* `:lh` *lh* `:rf` *rf* `:rg` *rg* `:rh` *rh* `:orgn` *orgn*`)` *[Method]*
> The keyword arguments are:
- *lf*, the  object of type `morphism` representing the morphism $f$ of the reduction $\rho_1$.
- *lg*, the  object of type `morphism`  representing the morphism $g$ of the reduction $\rho_1$.
- *lh*, the object of type  `morphism` representing the morphism $h$ of the reduction $\rho_1$.
- *rf*, the object of type `morphism` representing the morphism $f$ of the reduction $\rho_2$.
- *rg*, the object of type `morphism` representing the morphism $g$ of the reduction $\rho_2$.
- *rh*, the object of type `morphism` representing the morphism $h$ of the reduction $\rho_2$.
- *org*, the comment list.

All the needed information is contained in the morphism structures.
In both cases, the method  pushes the created instance onto the list `*hmeq-list*`.

### Useful macros and functions

`(cat-init)` *[Function]*
> Clear among others, the list `*hmeq-list*`, list of user created homotopy equivalences and reset the global counter to 1.

`(hmeq` *n*`)` *[Function]*
> Retrieve in the list `*hmeq-list*` the  homotopy equivalence instance whose identification is $n$. If it does not exist, return `NIL`.

`(lbcc` *hmeq* `&rest` *args*`)` *[Macro]*
> With only one argument (a homotopy equivalence *hmeq*) this macro selects the left bottom chain complex of the homotopy equivalence. Otherwise, it applies  the differential of the left bottom chain complex of *hmeq* on the arguments *args*, (either *degree generator* or *cmb*).

`(rbcc` *hmeq* `&rest` *args*`)` *[Macro]*
> With only one argument (a homotopy equivalence *hmeq*) this macro selects the right bottom chain complex of the homotopy equivalence. Otherwise, it applies  the differential of the right  bottom chain complex of *hmeq* on the arguments *args*, (either *degree generator* or *cmb*).

`(lf` *hmeq* `&rest` *args*`)` *[Macro]*
> With only one argument (a homotopy equivalence *hmeq*) this macro selects the morphism $f$ of the left reduction of *hmeq*. Otherwise, it applies  the morphism $f$ of the left reduction of *hmeq* on the arguments *args*, (either *degree generator* or *cmb*).

`(lg` *hmeq* `&rest` *args*`)` *[Macro]*
> With only one argument (a homotopy equivalence *hmeq*) this macro selects the morphism $g$ of the left reduction of *hmeq*. Otherwise, it applies  the morphism $g$ of the left reduction of *hmeq* on the arguments *args*, (either *degree generator* or *cmb*).

`(lh` *hmeq* `&rest` *args*`)` *[Macro]*
> With only one argument (a homotopy equivalence *hmeq*) this macro selects the morphism $h$ of the left reduction of *hmeq*. Otherwise, it applies  the morphism $h$ of the left reduction of *hmeq* on the arguments *args*, (either *degree generator* or *cmb*).

`(rf` *hmeq* `&rest` *args*`)` *[Macro]*
> With only one argument (a homotopy equivalence *hmeq*) this macro selects the morphism $f$ of the right reduction of *hmeq*. Otherwise, it applies  the morphism $f$ of the right reduction of *hmeq* on the arguments *args*, (either *degree generator* or *cmb*).

`(rg` *hmeq* `&rest` *args*`)` *[Macro]*
> With only one argument (a homotopy equivalence *hmeq*) this macro selects the morphism $g$ of the right reduction of *hmeq*. Otherwise, it applies  the morphism $g$ of the right reduction of *hmeq* on the arguments *args*, (either *degree generator* or *cmb*).

`(rh` *hmeq* `&rest` *args*`)` *[Macro]*
> With only one argument (a homotopy equivalence *hmeq*) this macro selects the morphism $h$ of the right reduction of *hmeq*. Otherwise, it applies  the morphism $h$ of the right reduction of *hmeq* on the arguments *args*, (either *degree generator* or *cmb*).

`(trivial-hmeq` *chcm*`)` *[Function]*
> Build the trivial homotopy equivalence involving only the chain complex *chcm*. In that case,
the 3 chain complexes $C$, $EC$ and $\hat C$ are  the chain complex *chcm* itself and the reductions $\rho_1$ and $\rho_2$ are, of course, the trivial reductions on *chcm* built by the statement `(trivial-rdct chcm)`.

## The perturbation lemma machinery

For a good understanding of the lisp functions involved in the machinery in question,
we recall the *perturbation lemma* (**Ronald Brown**. *The twisted Eilenberg--Zilber theorem*.
Celebrazioni Arch. Secolo XX Simp. Top., 1967, pp 34-37.). In the basic (resp. trivial) perturbation lemma, the given perturbation concerns the **top** (resp. **bottom**) chain complex.

**Theorem 1 (Basic perturbation lemma)** - Let
$\rho=(\hat{C},C,f,g,h)$ a reduction and $\hat{\delta}$ a *perturbation* of $d_{\hat{C}}$, that is, an operator defined on $\hat{C}$ of degree -1 satisfying the relation $(d_{\hat{C}}+\hat{\delta})\circ(d_{\hat{C}}+\hat{\delta})=0$. Furthermore, the composite function $h\circ\hat{\delta}$ is assumed *locally nilpotent*, that is, $\forall x \in \hat{C}$, $(h\circ\hat{\delta})^nx=0$, for $n$ sufficiently large. Then a new reduction $\rho'=(\hat{C}',C',f',g',h')$ can be constructed where:
1. $\hat{C}'$ is the chain complex obtained from $C$ by replacing the old differential $d_{\hat{C}}$ by $(d_{\hat{C}}+\hat{\delta}),$
2. the new chain complex $C'$ is obtained from the chain complex $C$ only by replacing  the old differential $d_C$ by $(d_C + \delta),$ with $\delta = f \circ {\hat \delta} \circ \phi \circ g =
               f \circ \psi \circ {\hat \delta} \circ g,$
3. $f'= f \circ \psi = f \circ (Id - {\hat \delta} \circ \phi \circ h),$
4. $g'= \phi \circ g,$
5. $h'= \phi \circ h = h \circ \psi$,
with 
$$
\phi=\sum_{i=0}^{\infty}{(-1)^i(h \circ  \hat{\delta})^i}
$$
and
$$
\psi=\sum_{i=0}^{\infty}{(-1)^i(\hat{\delta}\circ h)^i} = Id - {\hat \delta} \circ \phi \circ h,
$$
the convergence being ensured by the locally nilpotency of $h \circ \hat{\delta}$ and $\hat{\delta} \circ h$.

**Theorem 2 (Trivial perturbation lemma)** - Let
$\rho=(\hat{C},C,f,g,h)$ a reduction and ${\check\delta}$ a *perturbation* of $d_C$, that is, an operator defined on $C$ of degree -1 satisfying the relation $(d_C+{\check\delta})\circ (d_C+{\check\delta})=0$. Then a new reduction $\rho'=(\hat{C}',C',f',g',h')$ can be constructed where:
1. $\hat{C}'$ is the chain complex obtained from $C$ by replacing the old differential $d_{\hat{C}}$ by $(d_{\hat{C}}+ g \circ {\check\delta} \circ f),$
2. the new chain complex $C'$ is obtained from the chain complex $C$ only by replacing  the old differential $d_C$ by $(d_C +{\check \delta}),$ 
3. $f'= f,$
4. $g'= g,$
5. $h'= h.$

### Useful functions related to the perturbation lemma

The functions implementing the perturbation-lemma are actually the heart of the program. They allow us
to implement the effective versions of the classical spectral sequences (Serre, Eilenberg-Moore, ...). 

`(add` *rdct perturbation*`)` *[Method]*
> Build a new reduction from the  object of type `reduction`, *rdct* and 
the  object of type `morphism`,  *perturbation*, applying the formulas given in the perturbation lemma. In fact, this method calls  one or the other of the two following functions according as the perturbation is on the top or on the bottom chain complex and returns the values of the called function.

`(basic-perturbation-lemma` *reduction top-perturbation*`)` *[Function]*
> Return a double value: the perturbed reduction and the computed bottom-perturbation (a morphism)
$f \circ {\hat \delta}\circ  \phi \circ g$. Of course, to avoid unuseful computations when the morphisms are effectively applied, the function takes in account the simplifications involved for instance when some morphisms are the null morphism.

`(easy-perturbation-lemma` *reduction bottom-perturbation*`)` *[Function]*
> Return a double value: the perturbed reduction and the computed top-perturbation (a morphism) $g \circ {\check \delta} \circ f$. Of course, to avoid useless computation when the morphisms will be applied, the function takes in account the simplifications involved for instance when some morphisms are the null morphism.

`(special-bpl` *reduction top-perturbation*`)` *[Function]*
> This function is analogous to `basic-perturbation-lemma` and is used in some special cases when the morphism $g$ is invariant.

`(special-bpl2` *reduction top-perturbation*`)` *[Function]*
> [analogous to special-bpl when $f$ is invariant.]

`(bpl-*-sigma` *homotopy perturbation*`)` *[Function]*
> Construct the principal series $\phi=\sum_{i=0}^{\infty}{(-1)^i(h \circ  \hat{\delta})^i}$ of the basic perturbation lemma.

`(add` *hmeq lb-perturbation*`)` *[Method]*
> Build a homotopy equivalence from the `homotopy-equivalence` *hmeq* and  the `morphism` *lb-perturbation*, a perturbation  of the differential of the left bottom chain complex of *hmeq*. From *hmeq*:
$$
\begin{aligned}
  & \qquad\hat{C} & \\
 \rho_1\swarrow\nearrow   & & \nwarrow\searrow \rho_2 \\
C\quad  & & EC
\end{aligned}
$$
the method `add` builds the new homotopy equivalence:
$$
\begin{aligned}
  & \qquad\hat{C}' & \\
 \rho'_1\swarrow\nearrow   & & \nwarrow\searrow \rho'_2 \\
C'\quad  & & EC'
\end{aligned}
$$
where the  construction steps are the following:
- The reduction $\rho'_1$ is computed with the  method `add` for a reduction with arguments: the left  reduction of *hmeq* and the perturbation *lb-perturbation*. The trivial perturbation lemma is applied. As a bonus, this returns also the top perturbation, say $\delta$ to be applied to $\hat{C}$.
- The reduction $\rho'_2$ is computed with the previous method `add` with arguments: the right reduction of *hmeq* and the top perturbation $\delta$ just computed. That time, the basic perturbation lemma is applied.
- The function `build-hmeq` is called with parameters $\rho'_1$ and $\rho'_2$.

#### Example

We shall apply these methods and functions in the following chapters. Let us show on a trivial example
how this machinery works. First, we build a trivial homotopy equivalence on $\Delta^4$. Then we build a new one by perturbing the differential of the left bottom chain complex with its opposite. The last call shows how this perturbation has been propagated.

In [19]:
(def hmeq (trivial-hmeq (cdelta 4)))

[K66 Homotopy-Equivalence K30 <= K30 => K30]

In [20]:
(setf hmeq (add hmeq (opps (dffr (cdelta 4)))))

[K74 Homotopy-Equivalence K69 <= K69 => K69]

In [21]:
(gnrt-? (dffr (rbcc hmeq)) 3 '(0 1 2 3))


----------------------------------------------------------------------{CMBN 2}
------------------------------------------------------------------------------


## Bicones

Given 3 chain complexes $B$, $C$ and $D$, together with 2 chain homomorphisms $f_1$ and $f_2$ of degree $0$ (commuting with the differentials), as shown in the following diagram, a bicone on these chain complexes is a chain complex, $BCN(f_1,f_2)$ where the $n$-th chain group $[BCN(f_1,f_2)]$ is $B_n \oplus C_{n+1} \oplus D_n$.
$$
\begin{aligned}
B & & & \qquad D & \\
& f_1 \searrow & & \swarrow f_2 & \\
& & C \qquad& &
\end{aligned}
$$

### Representation of a combination in a bicone

To distinguish to which chain complex belongs a generator in a combination of a bicone, the following convention has been adopted: if `gb` is a generator of any degree of $B$, it will be represented in the bicone by the list `(:BcnB gb)` and printed as `<BcnB gb>`. The symbols for $C$ and $D$ are respectively `:BcnC` and `:BcnD`. The 3 macros `bcnb`, `bcnc` and `bcnd` may be used to build such a generator. The building function for combinations, `cmbn`, may be used in the following way:

In [22]:
(def comb-bic (cmbn 3 2 (bcnb 'b1) 4 (bcnb 'b1) 6 (bcnb 'b3)
                      3 (bcnb 'c1) 5 (bcnb 'c1) 7 (bcnb 'd1)))


----------------------------------------------------------------------{CMBN 3}
<2 * <BcnB B1>>
<4 * <BcnB B1>>
<6 * <BcnB B3>>
<3 * <BcnB C1>>
<5 * <BcnB C1>>
<7 * <BcnB D1>>
------------------------------------------------------------------------------


In [23]:
(cmbn-list comb-bic)

((2 . <BcnB B1>) (4 . <BcnB B1>) (6 . <BcnB B3>) (3 . <BcnB C1>) (5 . <BcnB C1>) (7 . <BcnB D1>))

### Useful functions and macros for bicones

`(bcnb` *gnrt*`)` *[Macro]*
> Build the representation of the generator *gnrt* belonging to the chain complex $B$.

`(bcnc` *gnrt*`)` *[Macro]*
> Build the representation of the generator *gnrt* belonging to the chain complex $C$.

`(bcnd` *gnrt*`)` *[Macro]*
> Build the representation of the generator *gnrt* belonging to the chain complex $D$.

`(make-bicn-cmbn` *cmbnb cmbnc cmbnd*`)` *[Function]*
> Build a bicone combination from the 3 combinations *cmbnb*, *cmbnc* and *cmbnd* belonging respectively and in that order to $B$, $C$ and $D$. The degrees of the combinations *cmbnb* and *cmbnd* must be the same, say $n$. The degree of *cmbnc* must be $n+1$. The degree of the new created combination is $n$.

`(bicn-cmbn-cmbnb` *cmbn*`)` *[Function]*
> Extract from the bicone combination of degree $n$, *cmbn*, the combination relative to $B$ as a legal combination of degree $n$ in $B$. If there is no $B$-component, return the null combination of degree $n$.

`(bicn-cmbn-cmbnc` *cmbn*`)` *[Function]*
> Extract from the bicone combination of degree $n$, *cmbn*, the combination relative to $C$ as a legal combination of degree $n+1$ in $C$. If there is no $C$-component, return the null combination of degree $n+1$.

`(bicn-cmbn-cmbnd` *cmbn*`)` *[Function]*
> Extract from the bicone combination of degree $n$, *cmbn*, the combination relative to $D$ as a legal combination of degree $n$ in $D$. If there is no $D$-component, return the null combination of degree $n$

`(dispatch-bicn-cmbn` *cmbn*`)` *[Function]*
> Give the 3-values result made of the 3 combinations components of the bicone combination *cmbn*. These combinations are valid combinations in their respective chain complexes.

#### Examples

In [24]:
(def comb-b (cmbn 3 2 'b1 4 'b2 6 'b3))


----------------------------------------------------------------------{CMBN 3}
<2 * B1>
<4 * B2>
<6 * B3>
------------------------------------------------------------------------------


In [25]:
(def comb-c (cmbn 4 3 'c1 5 'c2))


----------------------------------------------------------------------{CMBN 4}
<3 * C1>
<5 * C2>
------------------------------------------------------------------------------


In [26]:
(def comb-d (cmbn 3 7 'd1))


----------------------------------------------------------------------{CMBN 3}
<7 * D1>
------------------------------------------------------------------------------


In [27]:
(def comb-bic (make-bicn-cmbn comb-b comb-c comb-d))


----------------------------------------------------------------------{CMBN 3}
<2 * <BcnB B1>>
<4 * <BcnB B2>>
<6 * <BcnB B3>>
<3 * <BcnC C1>>
<5 * <BcnC C2>>
<7 * <BcnD D1>>
------------------------------------------------------------------------------


In [28]:
(bicn-cmbn-cmbnb comb-bic)


----------------------------------------------------------------------{CMBN 3}
<2 * B1>
<4 * B2>
<6 * B3>
------------------------------------------------------------------------------


In [29]:
(bicn-cmbn-cmbnc comb-bic)


----------------------------------------------------------------------{CMBN 4}
<3 * C1>
<5 * C2>
------------------------------------------------------------------------------


In [30]:
(bicn-cmbn-cmbnd comb-bic)


----------------------------------------------------------------------{CMBN 3}
<7 * D1>
------------------------------------------------------------------------------


In [31]:
(multiple-value-bind (b c d)
    (dispatch-bicn-cmbn comb-bic)
    (list b c d))

(
----------------------------------------------------------------------{CMBN 3}
<2 * B1>
<4 * B2>
<6 * B3>
------------------------------------------------------------------------------
 
----------------------------------------------------------------------{CMBN 4}
<3 * C1>
<5 * C2>
------------------------------------------------------------------------------
 
----------------------------------------------------------------------{CMBN 3}
<7 * D1>
------------------------------------------------------------------------------
)

## Construction of a bicone from 2 reductions

From the slots of the respective chain complexes $B$, $C$ and $D$, it is possible to build the bicone chain complex. The 3 essential functions are the following:

`(bicone-cmpr cmprb cmprc cmprd)` *[Function]*
> From the 3 comparison functions *cmprb*, *cmprc* and *cmprd*, build a comparison function adequate to compare the generators as represented in the bicone.

`(bicone-basis basisb basisc basisd)` *[Function]*
> From the 3 basis function *basisb*, *basisc* and *basisd*, build a basis function for the bicone. If at least one of the chain complex component of the bicone is *locally effective*, the function returns the symbol `:locally-effective`.

`(bicone-intr-dffr cmprc dffrb dffrc dffrd f1 f2)` *[Function]*
Define the differential in the bicone according to the formula:
$$
d(cb, cc, cd)= (d_B(cb), f_1(cb) + f_2(cd) - d_C(cc), d_D(cd)),
$$
where the notation is self explanatory. One sees that one needs the comparison operator *cmprc* of $C$, to order correctly the second combination of the resulting triple.

Let us consider now 2 reductions $(\hat C_1, C_1, f1, g1, h1)$ and $(\hat C_2, C_2, f2, g2, h2)$
in which $C_1 = C_2$. It is possible to define a bicone, with the identification $\hat C_1=B, C_1=C_2=C, \hat C_2=D$. This is realised by the following function:

`(bicone rdct1 rdct2)` *[Function]*
> Build the bicone chain complex from the objects ot type `reduction` *rdct1* and *rdct2*.
The bicone is built with the following identification: $B$ is the top chain complex of *rdct1*,
$D$ is the top chain complex of *rdct2*. The bottom chain complexes of both *rdct1* and *rdct2*
must be the same and $C$ is  that chain complex. Of course, both chain morphisms $f_1$ and
$f_2$ are respectively the morphism $f$ of the reductions. The construction of the bicone is realised
by a call to the building function `build-chcm` using the 3 functions `bicone-cmpr`, `bicone-basis` and `bicone-intr-dffr` with arguments coming from *rdct1* an *rdct2*. 
The slot `bsgn` (base point) is let undefined and the strategy is by combination.

#### Example

Let us take again the example with the standard simplex $\Delta^n$. The following function defines an effective version of ${\cal C}_*(\Delta^n)$, so we may ask for the basis in some dimensions.

In [32]:
(cat-init)


---done---

NIL

In [33]:
(defun cdelta (dmns)
    (build-chcm
     :cmpr #'l-cmpr
     :basis #'(lambda (n)
                      (mapcar #'dlop-int-ext (funcall (delta-n-basis dmns) n)))
     :bsgn '(0)
     :intr-dffr
         #'(lambda (degr gmsm)
            (make-cmbn
             :degr (1- degr)
             :list (do ((rslt +empty-list+
                              (cons (cons sign
                                          (append
                                           (subseq gmsm 0 nark)
                                           (subseq gmsm (1+ nark))))
                                    rslt))
                        (sign 1 (- sign))
                        (nark 0 (1+ nark)))
                       ((> nark degr) rslt))))
     :strt :gnrt
     :orgn `(Effective version of C_* delta ,dmns)))

REDEFINITION-WITH-DEFUN: 
  redefining CAT::CDELTA in DEFUN


CDELTA

In [34]:
(def delta3 (cdelta 3))

[K1 Chain-Complex]

In [35]:
(basis delta3 0)

((0) (1) (2) (3))

In [36]:
(basis delta3 1)

((0 1) (0 2) (1 2) (0 3) (1 3) (2 3))

In [37]:
(basis delta3 2)

((0 1 2) (0 1 3) (0 2 3) (1 2 3))

In [38]:
(basis delta3 3)

((0 1 2 3))

In [39]:
(basis delta3 4)

NIL

Using the function `make-rdct` defined in previous example which builds a reduction between 
$\Delta^m$ and $\Delta^n$, we may create a bicone chain complex with the function `bicone` and ask for basis in some dimensions.

In [40]:
(def bic (bicone (make-rdct 3 2) (make-rdct 4 2)))

[K15 Chain-Complex]

In [41]:
(basis bic 0)

(<BcnB (0)> <BcnB (1)> <BcnB (2)> <BcnB (3)> <BcnC (0 1)> <BcnC (0 2)> <BcnC (1 2)> <BcnD (0)> <BcnD (1)> <BcnD (2)> ...)

In [42]:
(basis bic 1)

(<BcnB (0 1)> <BcnB (0 2)> <BcnB (1 2)> <BcnB (0 3)> <BcnB (1 3)> <BcnB (2 3)> <BcnC (0 1 2)> <BcnD (0 1)> <BcnD (0 2)> <BcnD (1 2)> ...)

In [43]:
(basis bic 4)

(<BcnD (0 1 2 3 4)>)

We verify the validity of the differential in the bicone `bic`.

In [44]:
(? bic (cmbn 2 3 (bcnb '(0 1 3)) 4 (bcnc '(0 1 2 3)) 5 (bcnd '(0 1 4))))


----------------------------------------------------------------------{CMBN 1}
<3 * <BcnB (0 1)>>
<-3 * <BcnB (0 3)>>
<3 * <BcnB (1 3)>>
<12 * <BcnC (0 1 2)>>
<-4 * <BcnC (0 1 3)>>
<4 * <BcnC (0 2 3)>>
<-4 * <BcnC (1 2 3)>>
<5 * <BcnD (0 1)>>
<-5 * <BcnD (0 4)>>
<5 * <BcnD (1 4)>>
------------------------------------------------------------------------------


In [45]:
(? bic *)


----------------------------------------------------------------------{CMBN 0}
------------------------------------------------------------------------------


## Composition of homotopy equivalences

Let us consider 2 homotopy equivalences, as in the following diagram.
$$
\begin{aligned}
& \qquad B & & & & \qquad D \\
  \rho_1 \swarrow\nearrow  & & \nwarrow\searrow \rho_2 & 
& \rho'_1\swarrow\nearrow  & & \nwarrow\searrow\rho'_2 \\
A \quad &                & C &  & C \quad & & E \quad
\end{aligned}
$$
where the right bottom complex of the first one is the same as the left bottom complex of the second. We may use then the bicone concept to build a bicone chain complex with $B$, $C$ and $D$, say $BCN$, and finally build a new homotopy equivalence between $A$, $BCN$ and $E$.

$$
\begin{aligned}
  & \qquad BCN & \\
 \rho''_1 \swarrow\nearrow  & & \nwarrow\searrow \rho''_2 \\
A \quad & &  E\quad
\end{aligned}
$$

This is realized by the method `cmps`.

`(cmps` *hmeq1 hmeq2*`)` *[Method]*
> Build a homotopy equivalence by composition of both homotopy equivalences *hmeq1*, *hmeq2*. The system controls the validity of the composition.

### Example

Starting with a very simple chain complex and the trivial homotopy equivalence on the chain complex, we verify the correctness of various compositions. The chain complex `c` has only one vertex `(a)` in every dimension and the differential is the null morphism.

In [46]:
(cat-init)


---done---

NIL

In [47]:
(def c (build-chcm
            :cmpr #'s-cmpr
            :basis #'(lambda (dmns) (declare (ignore dmns)) '(a))
            :bsgn 'a
            :intr-dffr #'zero-intr-dffr
            :strt :cmbn
            :orgn '(c)))

[K1 Chain-Complex]

We build the trivial homotopy equivalence on `c` and we compose it with itself.

In [48]:
(def h1 (trivial-hmeq c))

[K6 Homotopy-Equivalence K1 <= K1 => K1]

In [49]:
(def h2 (cmps h1 h1))

[K17 Homotopy-Equivalence K1 <= K7 => K1]

We verify the coherency of the morphisms in the left and right reduction of `h2`. The combination `*tc*` must belong to a bicone whereas `*bc*` belongs to `c`.

In [50]:
(pre-check-rdct (lrdct h2))


---done---

NIL

In [51]:
(setf *tc* (cmbn 3 1 (bcnb 'a) 10 (bcnc 'a) 100 (bcnd 'a)))


----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB A>>
<10 * <BcnC A>>
<100 * <BcnD A>>
------------------------------------------------------------------------------


In [52]:
(setf *bc* (cmbn 3 1 'a))


----------------------------------------------------------------------{CMBN 3}
<1 * A>
------------------------------------------------------------------------------


In [53]:
(check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB A>>
<10 * <BcnC A>>
<100 * <BcnD A>>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * A>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 2}
------------------------------------------------------------------------------

Checking *DG-GD* = 0
Result: 
--------------

NIL

In [54]:
(pre-check-rdct (rrdct h2))


---done---

NIL

In [55]:
(check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB A>>
<10 * <BcnC A>>
<100 * <BcnD A>>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * A>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 2}
------------------------------------------------------------------------------

Checking *DG-GD* = 0
Result: 
--------------

NIL

We compose now `h2` with itself. We let `*bc*` unchanged, but `*tc*` is a little more complicated because the chain complexes $B$ and $D$  are themselves bicones.

In [56]:
(def h3 (cmps h2 h2))

[K65 Homotopy-Equivalence K1 <= K55 => K1]

In [57]:
(setf *tc* (cmbn 3 1 (bcnb (bcnb 'a)) 10 (bcnb (bcnc 'a)) 100 (bcnb (bcnd 'a))
                      1000 (bcnc 'a) 10000 (bcnd (bcnb 'a)) 5234 (bcnd (bcnc 'a))
                       223 (bcnd (bcnd 'a))))


----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB <BcnB A>>>
<10 * <BcnB <BcnC A>>>
<100 * <BcnB <BcnD A>>>
<1000 * <BcnC A>>
<10000 * <BcnD <BcnB A>>>
<5234 * <BcnD <BcnC A>>>
<223 * <BcnD <BcnD A>>>
------------------------------------------------------------------------------


In [58]:
(pre-check-rdct (lrdct h3))


---done---

NIL

In [59]:
(check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB <BcnB A>>>
<10 * <BcnB <BcnC A>>>
<100 * <BcnB <BcnD A>>>
<1000 * <BcnC A>>
<10000 * <BcnD <BcnB A>>>
<5234 * <BcnD <BcnC A>>>
<223 * <BcnD <BcnD A>>>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * A>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 2}
----------

NIL

In [60]:
(pre-check-rdct (rrdct h3))


---done---

NIL

In [61]:
(check-rdct-no-wait)


*TC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * <BcnB <BcnB A>>>
<10 * <BcnB <BcnC A>>>
<100 * <BcnB <BcnD A>>>
<1000 * <BcnC A>>
<10000 * <BcnD <BcnB A>>>
<5234 * <BcnD <BcnC A>>>
<223 * <BcnD <BcnD A>>>
------------------------------------------------------------------------------

*BC* => 
----------------------------------------------------------------------{CMBN 3}
<1 * A>
------------------------------------------------------------------------------

Checking *TDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *BDD* = 0
Result: 
----------------------------------------------------------------------{CMBN 1}
------------------------------------------------------------------------------

Checking *DF-FD* = 0
Result: 
----------------------------------------------------------------------{CMBN 2}
----------

NIL

#### Lisp files concerned in this chapter

`effective-homology.lisp`, `cones.lisp`.

[`classes.lisp`, `macros.lisp`, `various.lisp`].